In [1]:
import gym

#create a single game instance
env = gym.make("Taxi-v2")

#start new game
env.reset();

In [2]:
# display the game state
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [3]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

('initial observation code:', 133)
printing observation:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

('observations:', Discrete(500), 'n=', 500)
('actions:', Discrete(6), 'n=', 6)


In [4]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
('new observation code:', 153)
('reward:', -1)
('is game over?:', False)
printing new state:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


In [5]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

In [6]:
env.step(action_to_i['up'])
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


In [7]:
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


### Baseline: random search

In [8]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(6, size=500)

In [9]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies])
assert np.min(policies) == 0
assert np.max(policies) == n_actions-1
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

('Action frequencies over 10^4 samples:', array([ 0.1668264,  0.1667818,  0.166578 ,  0.166587 ,  0.166508 ,
        0.1667188]))
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [51]:
def sample_reward(env, policy, t_max=200):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    
    s = env.reset()
    total_reward = 0
    curStep = 0
    is_done = False
    while(not is_done and curStep != t_max):
        s, reward, is_done, _ = env.step(policy[s])
        total_reward += reward
        curStep += 1
    return total_reward

In [52]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
print("Looks good!")

generating 10^3 sessions...
Looks good!


In [53]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env,policy) for _ in range(n_times)]
    return float(np.mean(rewards))        

### Main loop

In [50]:
best_policy = None
best_score = -float('inf')

for i in range(100):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)
        print("Best policy:")

('New best score:', -953.3)
Best policy:
('New best score:', -883.1)
Best policy:


# Part II Genetic algorithm 

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [54]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    mask = np.random.choice([0, 1], len(policy1), p=[p,1 - p]) == 1
    resPolicy = np.zeros(len(policy1))
    resPolicy[mask] = policy1[mask]
    resPolicy[~mask] = policy2[~mask]
    return resPolicy

In [61]:
def mutation(policy, p=0.5):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(policy, get_random_policy(), p) #with p get_random_policy    

In [56]:
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [57]:

n_epochs = 250 #how many cycles to make
pool_size = 150 #how many policies to maintain
n_crossovers = 40 #how many crossovers to make on each step
n_mutations = 80 #how many mutations to make on each tick


In [62]:
np.random.seed(2341)
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)]
pool_scores = [evaluate(policy) for policy in pool]

initializing...


In [59]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])

In [22]:
#main loop
for epoch in range(n_epochs):
    print("Epoch %s:"%epoch)
    
    randNums = np.random.choice(range(len(pool)),  n_crossovers * 2, replace = False)
    crossovered = [crossover(pool[randNums[2*i]], pool[randNums[2*i + 1]]) for  i in range(n_crossovers)]
   # crossovered = <crossover random guys from pool, n_crossovers total>
    randNums = np.random.choice(range(len(pool)),  n_mutations, replace = False)
    mutated =  [mutation(pool[randNums[i]]) for  i in range(n_mutations)]
    
    #add new policies to the pool
    pool += crossovered + mutated
    pool_scores = [evaluate(pool[i]) for  i in range(len(pool))]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])

Epoch 0:
('best score:', -457.12)
Epoch 1:
('best score:', -412.66)
Epoch 2:
('best score:', -431.2)
Epoch 3:
('best score:', -369.19)
Epoch 4:
('best score:', -386.74)
Epoch 5:
('best score:', -367.48)
Epoch 6:
('best score:', -332.83)
Epoch 7:
('best score:', -306.19)
Epoch 8:
('best score:', -377.74)
Epoch 9:
('best score:', -333.1)
Epoch 10:
('best score:', -323.65)
Epoch 11:
('best score:', -359.11)
Epoch 12:
('best score:', -333.1)
Epoch 13:
('best score:', -323.65)
Epoch 14:
('best score:', -333.01)
Epoch 15:
('best score:', -288.28)
Epoch 16:
('best score:', -315.1)
Epoch 17:
('best score:', -324.37)
Epoch 18:
('best score:', -288.28)
Epoch 19:
('best score:', -332.2)
Epoch 20:
('best score:', -332.56)
Epoch 21:
('best score:', -315.19)
Epoch 22:
('best score:', -315.1)
Epoch 23:
('best score:', -314.11)
Epoch 24:
('best score:', -306.19)
Epoch 25:
('best score:', -314.2)
Epoch 26:
('best score:', -296.47)
Epoch 27:
('best score:', -270.46)
Epoch 28:
('best score:', -261.64)
Ep

('best score:', -100.0)
Epoch 235:
('best score:', -100.0)
Epoch 236:
('best score:', -100.0)
Epoch 237:
('best score:', -100.0)
Epoch 238:
('best score:', -100.0)
Epoch 239:
('best score:', -100.0)
Epoch 240:
('best score:', -100.0)
Epoch 241:
('best score:', -100.0)
Epoch 242:
('best score:', -100.0)
Epoch 243:
('best score:', -100.0)
Epoch 244:
('best score:', -100.0)
Epoch 245:
('best score:', -100.0)
Epoch 246:
('best score:', -100.0)
Epoch 247:
('best score:', -100.0)
Epoch 248:
('best score:', -100.0)
Epoch 249:
('best score:', -100.0)


Ссылка на фидбек по семинару: [link](https://docs.google.com/forms/d/e/1FAIpQLSf-08wFrEke6zKlysETYiqAjH5CRXtOKut5Q77Tr5rdVId7zA/)

In [63]:

n_epochs = 250 #how many cycles to make
pool_size = 150 #how many policies to maintain
n_crossovers = 40 #how many crossovers to make on each step
n_mutations = 80 #how many mutations to make on each tick


In [ ]:
#main loop
for epoch in range(n_epochs):
    print("Epoch %s:"%epoch)
    
    randNums = np.random.choice(range(len(pool)),  n_crossovers * 2, replace = False)
    crossovered = [crossover(pool[randNums[2*i]], pool[randNums[2*i + 1]]) for  i in range(n_crossovers)]
   # crossovered = <crossover random guys from pool, n_crossovers total>
    randNums = np.random.choice(range(len(pool)),  n_mutations, replace = False)
    mutated =  [mutation(pool[randNums[i]]) for  i in range(n_mutations)]
    
    #add new policies to the pool
    pool += crossovered + mutated
    pool_scores = [evaluate(pool[i]) for  i in range(len(pool))]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])

Epoch 0:
('best score:', -900.11)
Epoch 1:
('best score:', -828.74)
Epoch 2:
('best score:', -756.83)
Epoch 3:
('best score:', -846.2)
Epoch 4:
('best score:', -863.39)
Epoch 5:
('best score:', -792.02)
Epoch 6:
('best score:', -792.65)
Epoch 7:
('best score:', -881.48)
Epoch 8:
('best score:', -827.93)
Epoch 9:
('best score:', -792.65)
Epoch 10:
('best score:', -774.38)
Epoch 11:
('best score:', -685.1)
Epoch 12:
('best score:', -738.47)
Epoch 13:
('best score:', -720.65)
Epoch 14:
('best score:', -739.28)
Epoch 15:
('best score:', -791.12)
Epoch 16:
('best score:', -756.83)
Epoch 17:
('best score:', -720.38)
Epoch 18:
('best score:', -685.01)
Epoch 19:
('best score:', -685.37)
Epoch 20:
('best score:', -738.56)
Epoch 21:
('best score:', -737.84)
Epoch 22:
('best score:', -719.84)
Epoch 23:
('best score:', -541.19)
Epoch 24:
('best score:', -648.65)
Epoch 25:
('best score:', -702.47)
Epoch 26:
('best score:', -666.11)
Epoch 27:
('best score:', -648.56)
Epoch 28:
('best score:', -666.7